# LeNet-5 — Architektúra

Súbor: lenet5.ipynb

Program: Hospodárska informatika

Vypracovala:  Veronika Motúzová 

Bakalárska práca: Metódy hlbokého učenia pre detekciu meteorov v astronomických snímkach

Vedúci bakalárskej práce: doc. Ing. Peter Butka, PhD.

Konzultant: Ing. Viera Maslej Krešňáková

architektúra podľa: https://medium.com/datadriveninvestor/five-powerful-cnn-architectures-b939c9ddd57b

Import knižníc

In [2]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.layers import Conv2D, MaxPooling2D, Dropout, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
import array
from sklearn.metrics import confusion_matrix,classification_report
import itertools 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 

Načítanie dát

In [3]:
train = np.load('../data/1500_train.npz')
test = np.load('../data/1500_test.npz')

In [4]:
x_train = train['train_data'].reshape(train['train_data'].shape[0], 128, 128, 1)
x_test = test['test_data'].reshape(test['test_data'].shape[0], 128, 128, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [6]:
print(x_train.shape)
print(x_test.shape)

(16000, 128, 128, 1)
(16000, 128, 128, 1)


In [6]:
y_train = np_utils.to_categorical(train['train_targets'], 2)
y_test = np_utils.to_categorical(test['test_targets'], 2)

CNN LeNet-5

In [8]:
model = Sequential()

In [9]:
model.add(ZeroPadding2D((1,1),input_shape=(128, 128,1)))

In [10]:
#conv1
model.add(Conv2D(filters=20, kernel_size=5,padding='same', activation='relu'))

In [11]:
#pool1
model.add(MaxPooling2D())

In [12]:
#conv2
model.add(Conv2D(filters=50, kernel_size=5,padding='same', activation='relu'))

In [13]:
#pool2
model.add(MaxPooling2D())

In [14]:
#flatten
model.add(Flatten())

In [15]:
#dense1
model.add(Dense(500, activation='relu'))

In [16]:
model.add(Dense(2, activation='softmax'))

Adam Optimization

In [17]:
adam = Adam()

In [18]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Aby sa uložil model s najlepším výsledkom:

In [19]:
saved_model = "lenet5.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [20]:
f = model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_test, y_test), callbacks=[checkpoint])


Train on 16000 samples, validate on 16000 samples
Epoch 1/15
16000/16000 [==============================] - 295s 18ms/step - loss: 0.4529 - accuracy: 0.7551 - val_loss: 0.3504 - val_accuracy: 0.8444

Epoch 00001: val_accuracy improved from -inf to 0.84444, saving model to lenet5.hdf5
Epoch 2/15
16000/16000 [==============================] - 324s 20ms/step - loss: 0.3275 - accuracy: 0.8493 - val_loss: 0.3078 - val_accuracy: 0.8581

Epoch 00002: val_accuracy improved from 0.84444 to 0.85806, saving model to lenet5.hdf5
Epoch 3/15
16000/16000 [==============================] - 332s 21ms/step - loss: 0.2640 - accuracy: 0.8852 - val_loss: 0.2369 - val_accuracy: 0.9017

Epoch 00003: val_accuracy improved from 0.85806 to 0.90169, saving model to lenet5.hdf5
Epoch 4/15
16000/16000 [==============================] - 327s 20ms/step - loss: 0.2017 - accuracy: 0.9156 - val_loss: 0.2379 - val_accuracy: 0.9075

Epoch 00004: val_accuracy improved from 0.90169 to 0.90750, saving model to lenet5.hdf5


Načítanie modelu

In [7]:
print("Loading model....")
model = load_model('lenet5.hdf5')

Loading model....




Predikcia

In [8]:
y_pred = model.predict(x_test)
prediction_int = np.zeros_like(y_pred)
prediction_int[y_pred > 0.5] = 1

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.23742769477516412
Test accuracy: 0.9275000095367432


Confusion Matrix

In [10]:
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [11]:
print(matrix)
# TN / FN
# FP / TP

[[7556  444]
 [ 716 7284]]


Classification Report

In [12]:
print(classification_report(y_test, prediction_int))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93      8000
           1       0.94      0.91      0.93      8000

   micro avg       0.93      0.93      0.93     16000
   macro avg       0.93      0.93      0.93     16000
weighted avg       0.93      0.93      0.93     16000
 samples avg       0.93      0.93      0.93     16000



In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 130, 130, 20)      520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 65, 65, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 65, 65, 50)        25050     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 50)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)              